**Статистическая языковая модель**

Евгений Борисов <esborisov@sevsu.ru>

подбираем наиболее вероятное продолжение цепочки слов (NLTK model)

In [1]:
import re
import gzip

file_name = '../data/dostoevsky-besy-p2.txt.gz'

# загружаем текст ...
with gzip.open(file_name,'rt') as f:  
    text = f.read()[105:] # ...и выкидываем заголовок

print('символов:%i\n'%(len(text)))
print(text[:364].strip())

символов:465490

Теперь, когда уже все прошло, и я пишу хронику, мы уже знаем в чем дело; но тогда мы еще ничего не знали, и естественно, что нам представлялись странными разные вещи. По крайней мере мы со Степаном Трофимовичем в первое время заперлись и с испугом наблюдали издали. Я-то кой-куда еще выходил и по-прежнему приносил ему разные вести, без чего он и пробыть не мог.


In [2]:
from nltk import __version__ as nltk_version
print('nltk version:',nltk_version)

nltk version: 3.8.1


In [3]:
from random import sample

from nltk.tokenize import sent_tokenize as nltk_sentence_split
from nltk.tokenize import word_tokenize as nltk_tokenize_word

text = [ 
    nltk_tokenize_word(s) # разбиваем предложения на слова
    for s in nltk_sentence_split(text) # режем текст на отдельные предложения
]

print('предложений: %i\n'%(len(text)))

sample(text,2)

предложений: 5556



[['Они', 'посещали', 'там', 'многократно', '.'], ['Все', 'переглянулись', '.']]

In [4]:
from nltk.lm.preprocessing import padded_everygram_pipeline 

ngram_len_max = 2

# генерируем учебный датасет
train, vocab = padded_everygram_pipeline(ngram_len_max, text)

# собираем модель
from nltk.lm import MLE as Model 
# from nltk.lm import Laplace as Model

model = Model(ngram_len_max) 
model.fit(train, vocab)
print(len(model.vocab))

16634


In [5]:
from nltk.util import bigrams

# оцениваем насколько хорошо модель предсказывает слова из датасета
text_bigrams = [ ngram for s in text for ngram in bigrams(s) ]

display( f'perplexity: { model.perplexity(text_bigrams)}' )

'perplexity: 23.642337077815522'

In [6]:
# генерируем продолжения

from IPython.display import HTML

def crop_result(s):
    try:
        n = s.index('</s>')+1
    except ValueError:
        n = len(s)
    return s[:n]

def cstr(s, color='black'):
    return f'<font style="color:{color};">{s}</font>'

str_init_len = 9
words_predict = 5

for sentence in sample(text,20): # выбираем рандомно N предложений
    if len(sentence)<str_init_len+words_predict: continue # выкидываем очень короткие предложения
    sentence_ = sentence[:str_init_len] # берём начало предложения
    result = model.generate(words_predict, text_seed=sentence_) # генерируем возможные продолжения
    result = crop_result( result)
    display( HTML( cstr( ' '.join(sentence_)) + ' -> ' + cstr( ' '.join(result), color='red') ) ) 
